In [185]:
import pandas as pd
from gurobipy import *
from pyomo.environ import *

In [695]:
xls = pd.ExcelFile('miniset.xlsx', engine='openpyxl')
nombres= xls.sheet_names
df1 = pd.read_excel(xls, nombres[0])
df2 = pd.read_excel(xls, nombres[1])
df3= pd.read_excel(xls, nombres[2])
df4= pd.read_excel(xls, nombres[3])
df5= pd.read_excel(xls, nombres[4])

In [696]:
barcos_sheet = df1.to_dict("index")
compatibilidad_sheet = df2.to_dict()
cargos_sheet = df3.to_dict("index")
costos_transporte_sheet = df4.to_dict("index")
costos_transporte_dict = {(costos_transporte_sheet[i]['ID_Barco'],costos_transporte_sheet[i]['Puerto_Origen'],costos_transporte_sheet[i]['Puerto_Destino']) : {'Tiempo_Viaje_hrs':costos_transporte_sheet[i]['Tiempo_Viaje_hrs'], 'Costo_Viaje_libras':costos_transporte_sheet[i]['Costo_Viaje_libras']} for i in costos_transporte_sheet.keys()}
costos_tiempo_sheet = df5.to_dict("index")
costos_tiempo_dict = {(costos_tiempo_sheet[i]['ID_Barco'],costos_tiempo_sheet[i]['ID_Cargo']) : {'Tiempo_Origen_hrs':costos_tiempo_sheet[i]['Tiempo_Origen_hrs'], 'Costos_Origen_hrs':costos_tiempo_sheet[i]['Costos_Origen_hrs'], 'Tiempo_Destino_hrs':costos_tiempo_sheet[i]['Tiempo_Destino_hrs'], 'Costos_Destino_hrs':costos_tiempo_sheet[i]['Costos_Destino_hrs']} for i in costos_tiempo_sheet.keys()}

In [697]:
compatibilidad_sheet

{'ID_Barco': {0: 1}, 'ID_Cargo_Compatible': {0: 1}}

In [698]:
n_cargos = len(cargos_sheet)
n_cargos

1

In [699]:
V = [1+i for i in barcos_sheet.keys()] ## conjunto barcos
n_barcos = len(V)
print(V)

[1]


In [700]:
Kv= {} # conjunto capacidades de barcos
for i in barcos_sheet.keys():
    Kv[barcos_sheet[i]['ID_Barco']] =  barcos_sheet[i]['Capacidad']
Kv

{1: 16500}

In [701]:
# N^P
nodos_carga= [cargos_sheet[i]['ID_Cargo'] for i in cargos_sheet.keys()]
print(nodos_carga)

#N^D
nodos_descarga= [i+len(nodos_carga) for i in range(1,len(nodos_carga)+1)] ##conjunto nodos de descarga
print(nodos_descarga)

[1]
[2]


In [702]:
nodos = {}
for i in range(n_cargos):
    nodos[cargos_sheet[i]['ID_Cargo']] = {'ID_Cargo': cargos_sheet[i]['ID_Cargo'],
                                          'ID_Puerto': cargos_sheet[i]['ID_Puerto_Origen'],
                                          'Tamano': cargos_sheet[i]['Tamano'],
                                          'Costo_SPOT': cargos_sheet[i]['Costo_SPOT'],
                                          'LT': cargos_sheet[i]['LT_Carga'],
                                          'RT': cargos_sheet[i]['RT_Carga']}
    
    nodos[cargos_sheet[i]['ID_Cargo']+n_cargos] = {'ID_Cargo': cargos_sheet[i]['ID_Cargo'],
                                          'ID_Puerto': cargos_sheet[i]['ID_Puerto_Destino'],
                                          'Tamano': cargos_sheet[i]['Tamano'],
                                          'Costo_SPOT': cargos_sheet[i]['Costo_SPOT'],
                                          'LT': cargos_sheet[i]['LT_Descarga'],
                                          'RT': cargos_sheet[i]['RT_Descarga']}
o = {}
d = {}
for v in V:
    o[v] = n_cargos*2+v
    nodos[n_cargos*2+v] = {'Costo_SPOT': 0,
                           'ID_Cargo': 0,
                           'ID_Puerto': barcos_sheet[v-1]['ID_Puerto '],
                           'LT': barcos_sheet[v-1]['Tiempo_Inicio'],
                           'RT': barcos_sheet[v-1]['Tiempo_Inicio'],
                           'Tamano': 0}
    d[v] = n_cargos*2+n_barcos+v
    nodos[n_cargos*2+n_barcos+v] = {'Costo_SPOT': 0,
                                    'ID_Cargo': 0,
                                    'ID_Puerto': 0,
                                    'LT': 0,
                                    'RT': 100000,
                                    'Tamano': 0}
print(o) # nodos de origen
print(d) # nodos de destino
print(nodos) # todos los nodos con su información

{1: 3}
{1: 4}
{1: {'ID_Cargo': 1, 'ID_Puerto': 25, 'Tamano': 2368, 'Costo_SPOT': 354894, 'LT': 383, 'RT': 455}, 2: {'ID_Cargo': 1, 'ID_Puerto': 13, 'Tamano': 2368, 'Costo_SPOT': 354894, 'LT': 383, 'RT': 777}, 3: {'Costo_SPOT': 0, 'ID_Cargo': 0, 'ID_Puerto': 3, 'LT': 69, 'RT': 69, 'Tamano': 0}, 4: {'Costo_SPOT': 0, 'ID_Cargo': 0, 'ID_Puerto': 0, 'LT': 0, 'RT': 100000, 'Tamano': 0}}


In [703]:
compatibilidad = {}
for i in V:
    compatibilidad[i] = [int(j) for j in list(df2.loc[i-1])[1:] if j>0]
compatibilidad

Nv = {}
for i in V:
    Nv[i] = []
    for j in nodos.keys():
        if nodos[j]["ID_Cargo"] in compatibilidad[i] or (j == i+n_cargos*2 or j==i+n_cargos*2+len(V)): 
            Nv[i].append(j)
Nv

{1: [1, 2, 3, 4]}

In [704]:
"""import itertools
Av = {}
for v in V:
    Av[v] = list(itertools.permutations(Nv[v],2))
"""
Av={}
indices = []
for v in V:
    Av[v] = [] 
    for i in Nv[v]:
        for j in Nv[v]:
                Av[v].append((i,j))
                indices.append((i,j,v))
indices = tuplelist(indices)
print(Av[1])

[(1, 1), (1, 2), (1, 3), (1, 4), (2, 1), (2, 2), (2, 3), (2, 4), (3, 1), (3, 2), (3, 3), (3, 4), (4, 1), (4, 2), (4, 3), (4, 4)]


In [705]:
NP_v = {}
for v in V:
    NP_v[v] = [i for i in nodos_carga if i in Nv[v]]

ND_v = {}
for v in V:
    ND_v[v] = [i for i in nodos_descarga if i in Nv[v]]

print(NP_v)
print(ND_v)

{1: [1]}
{1: [2]}


In [706]:
LT_i = {i:nodos[i]["LT"] for i in nodos.keys()} #lower bounds ventana de tiempo por nodo
RT_i = {i:nodos[i]["RT"] for i in nodos.keys()} #upper bounds ventana de tiempo por nodo

print(LT_i)
print(RT_i)

{1: 383, 2: 383, 3: 69, 4: 0}
{1: 455, 2: 777, 3: 69, 4: 100000}


In [707]:
T_ijv={} # tiempo de entre i y j (tiempo de viaje + tiempo de operación en i)
for i in nodos.keys():
    T_ijv[i] = {} 
    for j in nodos.keys():
        T_ijv[i][j] = {} 
        for v in V:
            cargo_i= nodos[i]["ID_Cargo"]
            cargo_j = nodos[j]["ID_Cargo"]
            if i>n_cargos*2+len(V) or j>n_cargos*2+len(V):
                T_ijv[i][j][v] = 0
            else: 
                puerto_origen = nodos[i]["ID_Puerto"]
                puerto_destino = nodos[j]["ID_Puerto"]
                T_ijv[i][j][v] = costos_transporte_dict[v,puerto_origen,puerto_destino]['Tiempo_Viaje_hrs']
                if 0<i<=n_cargos and (i,j) in Av[v]: 
                    T_ijv[i][j][v] += costos_tiempo_dict[v,cargo_i]['Tiempo_Origen_hrs']
                if n_cargos*2>=i>n_cargos and (i,j) in Av[v]:
                    T_ijv[i][j][v] += costos_tiempo_dict[v,cargo_i]['Tiempo_Destino_hrs']
                
                # se les asigna tiempo de operación igual a 0 a los (i,j) que no pertenecen a Av
                # los (i,j) que no pertenecen a Av aún tienen el tiempo entre el puerto de i y j

In [708]:
C_ijv={} # costo de entre i y j (costo de viaje + costo de operación en i)
for i in nodos.keys():
    C_ijv[i] = {} 
    for j in nodos.keys():
        C_ijv[i][j] = {} 
        for v in V:
            cargo_i= nodos[i]["ID_Cargo"]
            cargo_j = nodos[j]["ID_Cargo"]
            if i>n_cargos*2+len(V) or j>n_cargos*2+len(V):
                C_ijv[i][j][v] = 0
            else: 
                puerto_origen = nodos[i]["ID_Puerto"]
                puerto_destino = nodos[j]["ID_Puerto"]
                C_ijv[i][j][v] = costos_transporte_dict[v,puerto_origen,puerto_destino]['Costo_Viaje_libras']
                if i==11 and j==2 and i==1:
                    print(C_ijv[i][j][v])
                if 0<i<=n_cargos and (i,j) in Av[v]:
                    C_ijv[i][j][v] += costos_tiempo_dict[v,cargo_i]['Costos_Origen_hrs']

                if n_cargos*2>=i>n_cargos and (i,j) in Av[v]:
                    C_ijv[i][j][v] += costos_tiempo_dict[v,cargo_i]['Costos_Destino_hrs']

In [709]:
Qj={i:nodos[i]["Tamano"] for i in nodos.keys()}
print("Qj: ",Qj)

print("\n")

Cs = {cargos_sheet[i]['ID_Cargo']:cargos_sheet[i]['Costo_SPOT'] for i in cargos_sheet.keys()}
print("Cs", Cs)

Qj:  {1: 2368, 2: 2368, 3: 0, 4: 0}


Cs {1: 354894}


In [710]:
nodos_index = [i for i in nodos.keys()]

In [711]:
1<2

True

In [712]:
def obj_rule(model):
    return sum(C_ijv[i][j][v]*model.x[i,j,v] for (i,j) in Av[v] for v in V)+sum(Cs[i]*model.y[i] for i in nodos_carga)

def const_1(model, i):
    return sum(model.x[i,j,v] for j in Nv[v] for v in V) + model.y[i] == 1

def const_2(model, v):
    return sum(model.x[o[v],j,v] for j in Nv[v]) == 1

def const_3(model, i, v):
    if i in Nv[v]:
        if i!=o[v] and i!=d[v]:
            return sum(model.x[i,j,v] for j in Nv[v])-sum(model.x[j,i,v] for j in Nv[v])==0
    return Constraint.Skip

def const_4(model, v):
    return sum(model.x[j,d[v],v] for j in Nv[v]) == 1

def const_5(model, i, j, v):
    if j in NP_v[v]:
        if (i,j) in Av[v]:
            return model.l[i,v]+Qj[j]-model.l[j,v]<=Kv[v]*(1-model.x[i,j,v])
    return Constraint.Skip

def const_6(model, i, j, v):
    if j in NP_v[v]:
        if (i,n_cargos+j) in Av[v]:
            return model.l[i,v]-Qj[j]-model.l[n_cargos+j,v]<=Kv[v]*(1-model.x[i,n_cargos+j,v])
    return Constraint.Skip

def const_7_ub(model, i, v):
    if i in NP_v[v]:
        return model.l[i,v]<=Kv[v]
    return Constraint.Skip

def const_7_lb(model, i, v):
    if i in NP_v[v]:
        return 0<=model.l[i,v]
    return Constraint.Skip

def const_8(model, i, j, v):
    if (i,j) in Av[v]:
        return model.t[i,v]+T_ijv[i][j][v]-model.t[j,v]<=(RT_i[i]+T_ijv[i][j][v])*(1-model.x[i,j,v])
    return Constraint.Skip

def const_9(model, i, v):
    if i in NP_v[v]:
        return sum(model.x[i,j,v] for j in Nv[v])-sum(model.x[n_cargos+i,j,v] for j in Nv[v])==0
    return Constraint.Skip

def const_10(model, i, v):
    if i in NP_v[v]:
        return model.t[i,v]+T_ijv[i][n_cargos+i][v]-model.t[n_cargos+i,v]<=0
    return Constraint.Skip
    
def const_11_ub(model, i, v):
    if i in Nv[v]:
        return model.t[i,v]<=RT_i[i]
    return Constraint.Skip

def const_11_lb(model, i, v):
    if i in Nv[v]:
        return LT_i[i]<=model.t[i,v]
    return Constraint.Skip


"""
"""

'\n'

In [728]:
model_1 = ConcreteModel()

In [729]:
model_1.x = Var(nodos_index, nodos_index, V, within=Binary)
model_1.y = Var(nodos_carga, within=Binary)
model_1.t = Var(nodos_index,V,domain=PositiveIntegers)
model_1.l = Var(nodos_index,V,domain=NonNegativeReals)

In [730]:
Kv[1]

16500

In [731]:
model_1.obj = Objective(rule=obj_rule)
model_1.const_1 = Constraint(nodos_carga, rule=const_1)
model_1.const_2 = Constraint(V, rule=const_2)
model_1.const_3 = Constraint(nodos_index, V, rule=const_3)
model_1.const_4 = Constraint(V, rule=const_4)
model_1.const_5 = Constraint(nodos_index, nodos_index, V, rule=const_5)
model_1.const_6 = Constraint(nodos_index, nodos_index, V, rule=const_6)
model_1.const_7_ub = Constraint(nodos_index, V, rule=const_7_ub)
model_1.const_7_lb = Constraint(nodos_index, V, rule=const_7_lb)
model_1.const_8 = Constraint(nodos_index, nodos_index, V, rule=const_8)
model_1.const_9 = Constraint(nodos_index, V, rule=const_9)
model_1.const_10 = Constraint(nodos_index, V, rule=const_10)
model_1.const_11_ub = Constraint(nodos_index, V, rule=const_11_ub)
model_1.const_11_lb = Constraint(nodos_index, V, rule=const_11_lb)
"""
"""

'\n'

In [732]:
solver_1 = SolverFactory('gurobi')
solver_1.solve(model_1)['Solver'][0]['Message']

'Model was solved to optimality (subject to tolerances), and an optimal solution is available.'

In [733]:
Kv[1]

16500

In [734]:
model_1.x.pprint()

x : Size=16, Index=x_index
    Key       : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1, 1) :     0 :   0.0 :     1 : False : False : Binary
    (1, 2, 1) :     0 :   1.0 :     1 : False : False : Binary
    (1, 3, 1) :     0 :   0.0 :     1 : False : False : Binary
    (1, 4, 1) :     0 :   0.0 :     1 : False : False : Binary
    (2, 1, 1) :     0 :   0.0 :     1 : False : False : Binary
    (2, 2, 1) :     0 :   0.0 :     1 : False : False : Binary
    (2, 3, 1) :     0 :   0.0 :     1 : False : False : Binary
    (2, 4, 1) :     0 :   1.0 :     1 : False : False : Binary
    (3, 1, 1) :     0 :   1.0 :     1 : False : False : Binary
    (3, 2, 1) :     0 :   0.0 :     1 : False : False : Binary
    (3, 3, 1) :     0 :   0.0 :     1 : False : False : Binary
    (3, 4, 1) :     0 :   0.0 :     1 : False : False : Binary
    (4, 1, 1) :     0 :   0.0 :     1 : False : False : Binary
    (4, 2, 1) :     0 :   0.0 :     1 : False : False : Binary
    (4, 3, 1) :     0 :   0.

In [735]:
model_1.y.pprint()

y : Size=1, Index=y_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   0.0 :     1 : False : False : Binary


In [736]:
model_1.t.pprint()

t : Size=4, Index=t_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     1 : 455.0 :  None : False : False : PositiveIntegers
    (2, 1) :     1 : 515.0 :  None : False : False : PositiveIntegers
    (3, 1) :     1 :  69.0 :  None : False : False : PositiveIntegers
    (4, 1) :     1 : 515.0 :  None : False : False : PositiveIntegers


In [737]:
model_1.l.pprint()

l : Size=4, Index=l_index
    Key    : Lower : Value   : Upper : Fixed : Stale : Domain
    (1, 1) :     0 : 16500.0 :  None : False : False : NonNegativeReals
    (2, 1) :     0 : 30632.0 :  None : False : False : NonNegativeReals
    (3, 1) :     0 :     0.0 :  None : False : False : NonNegativeReals
    (4, 1) :     0 :     0.0 :  None : False : False : NonNegativeReals


In [738]:
model_1.obj.expr()

103958.0

In [739]:
Qj[1]

2368